In [161]:
import numpy as np

# Basic operations 

In [162]:
class Operation:
    def __init__(self, input_nodes=[]):
        self.input_nodes = input_nodes
        self.output_nodes = []
        
        for node in input_nodes:
            node.output_nodes.append(self)
        
        _default_graph.operations.append(self)
        
    def compute(self):
        pass

In [163]:
class add(Operation):
    def __init__(self, x, y):
        super().__init__([x, y])
        
    def compute(self, x, y):
        self.inputs = [x, y]
        
        return x + y

In [164]:
class multiply(Operation):
    def __init__(self, x, y):
        super().__init__([x, y])
        
    def compute(self, x, y):
        self.inputs = [x, y]
        
        return x * y

In [165]:
class matmul(Operation):
    def __init__(self, x, y):
        super().__init__([x, y])
        
    def compute(self, x, y):
        self.inputs = [x, y]
        
        return x.dot(y)

# Placeholders, variables and graph

- Placeholders: An "empty" node that needs a value to be provided to compute output
- Variebles: Changeable parameter of Graph (e.g. weights)
- Graph: Global Variable connecting variables and placeholders operations

In [166]:
class Placeholder:
    def __init__(self):
        self.output_nodes = []
        
        _default_graph.placeholders.append(self)

In [167]:
class Variable:
    def __init__(self, value=None):
        self.value = value
        self.output_nodes = []
        
        _default_graph.variables.append(self)

In [168]:
class Graph:
    def __init__(self):
        self.operations = []
        self.placeholders = []
        self.variables = []
    
    def set_as_default(self):
        global _default_graph
        _default_graph = self

# Example

- $ z = ax + b $
- $ a = 10 $
- $ b = 1 $
- $ z = 10x + b $

In [169]:
g = Graph()
g.set_as_default()

In [170]:
a = Variable(10)
b = Variable(1)

In [171]:
x = Placeholder()

In [172]:
y = multiply(a, x)
z = add(y, b)

# Session
- Now that the Graph has all the nodes, we need to execute all operations within a Session
- We'll use a PostOrder Tree Traversal to make sure we execute the nodes in the correct order

In [173]:
def traverse_postorder(operation):
    nodes_postorder = []
    
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        
        nodes_postorder.append(node)
        
    recurse(operation)
    
    return nodes_postorder

In [174]:
class Session:
    def run(self, operation, feed_dict={}):
        nodes_postorder = traverse_postorder(operation)
        
        for node in nodes_postorder:
            if type(node) == Placeholder:
                node.output = feed_dict[node]
            elif type(node) == Variable:
                node.output = node.value
            else:
                # Operation
                node.inputs = [input_node.output for input_node in node.input_nodes]
                node.output = node.compute(*node.inputs)
            
            if type(node.output) == list:
                node.output = np.array(node.output)
            
        return operation.output

In [175]:
sess = Session()

In [176]:
sess.run(operation=z,feed_dict={x:10})

101

In [177]:
g = Graph()
g.set_as_default()

a = Variable([[10, 20], [30, 40]])
b = Variable([1, 1])
x = Placeholder()

z = add(matmul(a, x), b)

sess = Session()
sess.run(z, {x: 10})

array([[101, 201],
       [301, 401]])